In [1]:
import torch.nn as nn
import librosa
import numpy as np
import torch
import torch.utils.data
import matplotlib.pyplot as plt
import IPython.display as ipd

In [2]:
# from google.colab import files
# files.upload()


In [3]:
# files.upload()

In [4]:
s, sr=librosa.load('data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [5]:
s_abs = np.abs(S)
x_abs = np.abs(X)
s = torch.tensor(np.transpose(s_abs)).cuda()
x = torch.tensor(np.transpose(x_abs)).cuda()

In [6]:
print(s.shape[0])
print(x.shape)

2459
torch.Size([2459, 513])


In [7]:
x_frames = []
target_frames = []
for t in range(s.shape[0]-19):
    frame_x = s[t:t+20,:]
#     print(frame_x.shape)
    frame_y = x[t:t+20,:]
    x_frames.append(frame_x)
    target_frames.append(frame_y)

print(np.array(x_frames).shape)
print(np.array(target_frames).shape)

print(x_frames[2439].shape)

(2440,)
(2440,)
torch.Size([20, 513])


In [8]:
BATCH = 171
train_loader = torch.utils.data.DataLoader(x_frames, batch_size=BATCH)
test_loader = torch.utils.data.DataLoader(target_frames, batch_size=BATCH)
print(len(train_loader))
print(len(test_loader))

15
15


In [9]:
# test_iter = iter(train_loader)
# test_data = test_iter.next()
# print(test_data.shape)

In [10]:
# my_data = test_data.squeeze().cuda()
# print(my_data.shape)

In [11]:
def get_shape(input, filter, padding, stride):
  return (((input - filter + (2*padding)) / stride)) + 1

get_shape(512, 2,0, 1)

511.0

In [15]:
class NeuralNetwork(nn.Module):
  
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    
    
    self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=2, stride=1),
            nn.ReLU(),       
            nn.MaxPool2d(kernel_size=2, stride=1))

    
    self.conv2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=2, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1))
    
    
    self.fc1 = nn.Linear(in_features=int(260608), out_features=513)
    
  def forward(self, data):

    data = data[:,np.newaxis,:, :]
    print("first")
    print(data.shape)
    output = self.conv1(data)
    print("output from first")
    print(data.shape)
    output = self.conv2(output)
    print("output from second")
    print(output.shape)
    output = output.reshape(output.shape[0], -1)
    print("input to fully connected")
    print(output.shape)
    output = self.fc1(output)
#     final = torch.nn.functional.relu(output)
    final = output
    print("final shape")
    print(final.shape)
    return final

In [16]:
neural_network = NeuralNetwork().cuda()

In [17]:
loss_function= nn.MSELoss()
para = neural_network.parameters()
optimizer = torch.optim.Adam(params=para, lr=0.001)
loss_list = []
epochs = 500
for i in range(epochs):
    input_iter = iter(train_loader)
    target_iter = iter(test_loader)
    while True:
        try:
            input_set = input_iter.next()
            target_set = target_iter.next()
            input_set = input_set.cuda()
            target_set = target_set.cuda()
            optimizer.zero_grad()
            network_output  = neural_network(input_set)
            print("target set")
            print(target_set.shape)
            print("network shape")
            print(network_output.shape)
            loss = loss_function(network_output , target_set)
            loss.backward()
            optimizer.step()
        except StopIteration:
            break
    loss_list.append(loss.data.cpu().numpy())
print(loss.data)
plt.plot(range(epochs), loss_list)
plt.show()      

first
torch.Size([171, 1, 20, 513])
output from first
torch.Size([171, 1, 20, 513])
output from second
torch.Size([171, 32, 16, 509])
input to fully connected
torch.Size([171, 260608])
final shape
torch.Size([171, 513])
target set
torch.Size([171, 20, 513])
network shape
torch.Size([171, 513])


RuntimeError: The size of tensor a (171) must match the size of tensor b (20) at non-singleton dimension 1

In [ ]:
def test_audio(input_file_path, output_file_path):
    st, sr=librosa.load(input_file_path, sr=None)
    test = librosa.stft(st, n_fft=1024, hop_length=512)
    test_abs = torch.tensor(np.abs(test))
    test_abs = np.transpose(test_abs)
    test_loader = torch.utils.data.DataLoader(test_abs, batch_size=test_abs.shape[0])
    t_iter = iter(test_loader)
    with torch.no_grad():
        while True:
            try:
                data = t_iter.next()
#                 print(data.shape)
                data = data.cuda()
                output = neural_network(data)
            except StopIteration:
                break 
        spec = (test / np.abs(test)) * output.cpu().numpy().T
        spec_istft = librosa.istft(spec, hop_length=512)
        librosa.output.write_wav(output_file_path, spec_istft, sr)

In [ ]:
# testing on the audio 1
test_audio('data/test_x_01.wav', 'test_s_01_recons.wav')

In [ ]:
ipd.Audio('test_s_01_recons.wav')

In [ ]:
# testing on the audio 2
test_audio('data/test_x_02.wav', 'test_s_02_recons.wav')


In [ ]:
ipd.Audio('test_s_02_recons.wav')

In [ ]:
#checking how the original train file comes out to be
test_audio('data/train_dirty_male.wav', 'train_dirty_male_recons.wav')

In [ ]:
ipd.Audio('train_dirty_male_recons.wav')